In [1]:
import numpy as np
import gensim
import logging
import string
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM

from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file

Using Theano backend.


In [ ]:
url = 'https://github.com/jobaird/cs344/blob/master/AI%20Project/Hitchhikers_quotes.txt'
path = get_file('hitch.txt', origin=url)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [3]:
def print_input(input_file):
    f = open(input_file, "r")
    for i, line in enumerate(f):
        print(line)


def read_input(input_file):
    f = open(input_file, "r")
    for i, line in enumerate(f):

        yield gensim.utils.simple_preprocess(line)


def word2idx(word):
  return word_model.wv.vocab[word].index

def idx2word(idx):
  return word_model.wv.index2word[idx]

In [22]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)


def generate_next(text, num_generated=15):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)

def on_epoch_end(epoch, _):
  print('\nGenerating text after epoch: %d' % epoch)
  texts = [
      'the universe',
      'the guide',
      'arthur dent',
      'zaphod',
      'the answer',
  ]
  for text in texts:
    sample = generate_next(text)
    print('%s... -> %s' % (text, sample))

In [23]:
max_sentence_len = 40
with open(path) as file_:
  docs = file_.readlines()
sentences = [[word for word in doc.lower().translate(string.punctuation).split()[:max_sentence_len]] for doc in docs]
print('\nTraining word2vec...')
word_model = gensim.models.Word2Vec(sentences, size=40, min_count=1, window=5, iter=200)
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')
for word in ['universe', 'guide', 'arthur', 'zaphod', 'answer']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))

2019-05-13 10:06:18,752 : INFO : collecting all words and their counts
2019-05-13 10:06:18,753 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-13 10:06:18,759 : INFO : collected 2856 word types from a corpus of 8678 raw words and 371 sentences
2019-05-13 10:06:18,761 : INFO : Loading a fresh vocabulary
2019-05-13 10:06:18,774 : INFO : effective_min_count=1 retains 2856 unique words (100% of original 2856, drops 0)
2019-05-13 10:06:18,776 : INFO : effective_min_count=1 leaves 8678 word corpus (100% of original 8678, drops 0)
2019-05-13 10:06:18,789 : INFO : deleting the raw counts dictionary of 2856 items
2019-05-13 10:06:18,791 : INFO : sample=0.001 downsamples 47 most-common words
2019-05-13 10:06:18,792 : INFO : downsampling leaves estimated 6745 word corpus (77.7% of prior 8678)
2019-05-13 10:06:18,803 : INFO : estimated required memory for 2856 words and 40 dimensions: 2341920 bytes
2019-05-13 10:06:18,804 : INFO : resetting layer weights
2019-05-


Training word2vec...


2019-05-13 10:06:18,957 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-05-13 10:06:18,960 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-05-13 10:06:18,967 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-05-13 10:06:18,969 : INFO : EPOCH - 5 : training on 8678 raw words (6742 effective words) took 0.0s, 359865 effective words/s
2019-05-13 10:06:18,978 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-05-13 10:06:18,980 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-05-13 10:06:18,987 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-05-13 10:06:18,988 : INFO : EPOCH - 6 : training on 8678 raw words (6695 effective words) took 0.0s, 417247 effective words/s
2019-05-13 10:06:19,004 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-05-13 10:06:19,006 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-05-13

Result embedding shape: (2856, 40)
Checking similar words:
  universe -> beginning (0.78), everything"+ (0.77), concerning (0.75), as, (0.74), ways (0.74), life, (0.74), far: (0.74), such (0.72)
  guide -> hitchhiker's (0.96), galaxy (0.88), indeed. (0.82), quietly, (0.81), rim (0.80), require (0.80), definitive. (0.80), supplanted (0.79)
  arthur -> terrific," (0.85), cheered (0.83), sinking (0.81), shouted (0.77), "oh!" (0.77), low (0.77), "oh! (0.77), "pity," (0.76)
  zaphod -> relationship (0.80), experienced (0.80), tried (0.77), "yeah," (0.76), darkly, (0.75), wrest (0.74), beeblebrox (0.74), patting (0.74)
  answer -> “that (0.90), question (0.89), mutually (0.87), question, (0.86), life, (0.84), ultimate (0.84), means."+ (0.82), is."+ (0.79)


In [24]:
print('\nPreparing the data for LSTM...')
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)


Preparing the data for LSTM...
train_x shape: (371, 40)
train_y shape: (371,)


In [25]:
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


Training LSTM...


In [26]:
model.fit(train_x, train_y,
          batch_size=128,
          epochs=100,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/100
256/371 [===================>..........] - ETA: 0s - loss: 7.9606

/Users/John/PycharmProjects/cs344/venv/lib/python3.6/site-packages/theano/tensor/subtensor.py:2339: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


371/371 [==============================] - 0s 544us/step - loss: 7.9575

Generating text after epoch: 0
the universe... -> the universe except napkins with; second early indeed slid consequences during nervous gathering they'd ecstatic he's ford,
the guide... -> the guide thirty meat proved misty greatest meat organic spent having decided number produces major of. "really?"
arthur dent... -> arthur dent embrace, supermarket.+ "hmmm," leap systems."+ summary: wonder, gently time, galaxy, "listen, clunks almost useful close.+
zaphod... -> zaphod illuminated, idiot.+ small, glittering tragula—for laser-guided free hell happens, 'i drink. campaign, bridge. arthur, prospect
the answer... -> the answer again."+ legal question amen' solve sounded trillian.+ achieved tree crazy planning "no," prospect everything"+ "make
Epoch 2/100
371/371 [==============================] - 0s 514us/step - loss: 7.9035

Generating text after epoch: 1
the universe... -> the universe again?"+ already piece sense